In [32]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
# Load environment variables from the .secrets file
load_dotenv(".secrets")

True

In [33]:
# Load customized capital calls schedule from Excel file
call_schedule_df = pd.read_excel('data/PM_call_schedule.xlsx', sheet_name='Schedule')
call_schedule_df.rename(columns={'Asset_Class':'asset_class', 'Geo_Area':'geo_area'}, inplace=True)
call_schedule_df

,asset_class,geo_area,1,2,3,4,5,6,7,8,9,10,11,12
0,Private Equity,"US,APAC,ROW",0.20,0.30,0.30,0.20,0.00,0,0,0,0,0,0,0
1,Private Equity,EU,0.20,0.25,0.35,0.15,0.05,0,0,0,0,0,0,0
2,Venture Capital,"US,APAC,ROW",0.20,0.30,0.30,0.15,0.05,0,0,0,0,0,0,0
3,Venture Capital,EU,0.20,0.30,0.30,0.15,0.05,0,0,0,0,0,0,0
4,Co-Investments,"EU,US,APAC,ROW",0.50,0.50,0.00,0.00,0.00,0,0,0,0,0,0,0
5,Private Debt,"EU,US,APAC,ROW",0.20,0.40,0.40,0.00,0.00,0,0,0,0,0,0,0
6,Real Estate,"EU,US,APAC,ROW",0.20,0.40,0.40,0.00,0.00,0,0,0,0,0,0,0
7,Real Assets,"EU,US,APAC,ROW",0.20,0.40,0.40,0.00,0.00,0,0,0,0,0,0,0
8,Private Diversifiers,"EU,US,APAC,ROW",0.20,0.40,0.40,0.00,0.00,0,0,0,0,0,0,0
9,Infrastructure,"EU,US,APAC,ROW",0.20,0.40,0.40,0.00,0.00,0,0,0,0,0,0,0


In [34]:
# Melt the DataFrame to create 'Month' and 'Value' columns in long format
call_schedule_long_df = call_schedule_df.melt(id_vars=['asset_class', 'geo_area'], 
                                              var_name='call_year', value_name='call_value')

# Split the 'Geo_Area' column into multiple rows based on comma separation
call_schedule_long_df['geo_area'] = call_schedule_long_df['geo_area'].str.split(',')
call_schedule_long_df = call_schedule_long_df.explode('geo_area').reset_index(drop=True)
call_schedule_long_df.sort_values(by=['asset_class','geo_area','call_year'], inplace=True)
# Display the result
call_schedule_long_df

,asset_class,geo_area,call_year,call_value
10,Co-Investments,APAC,1,0.5
58,Co-Investments,APAC,2,0.5
106,Co-Investments,APAC,3,0.0
154,Co-Investments,APAC,4,0.0
202,Co-Investments,APAC,5,0.0
...,...,...,...,...
340,Venture Capital,US,8,0.0
388,Venture Capital,US,9,0.0
436,Venture Capital,US,10,0.0
484,Venture Capital,US,11,0.0


In [35]:
# Database connection parameters from environment variables
DB_NAME = os.getenv("AWS_RDS_DBNAME")
USER = os.getenv("AWS_RDS_USERNAME")
PASSWORD = os.getenv("AWS_RDS_PASSWORD")
HOST = os.getenv("AWS_RDS_HOST")
PORT = os.getenv("AWS_RDS_PORT")

# Connect to PostgreSQL database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)
cur = conn.cursor()
conn

<connection object at 0x0000022328401140; dsn: 'user=lbassetti password=xxx dbname=postgres host=sa-met-1.cna4g6g06njt.eu-central-2.rds.amazonaws.com port=5432', closed: 0>

In [36]:
# Insert DataFrame into SQL table
insert_query = sql.SQL("""
    INSERT INTO capital_call_schedule (asset_class, geo_area, call_year, call_value)
    VALUES (%s, %s, %s, %s)
    ON CONFLICT DO NOTHING  -- to avoid duplicate entries if re-running the code
""")

# Iterate through DataFrame rows with a progress bar and execute the insert query
for _, row in tqdm(call_schedule_long_df.iterrows(), total=len(call_schedule_long_df), desc="Inserting rows"):
    cur.execute(insert_query, (row['asset_class'], row['geo_area'], int(row['call_year']), float(row['call_value'])))

# Commit transaction
conn.commit()

# Close cursor and connection
cur.close()
conn.close()

Inserting rows: 100%|██████████| 576/576 [00:19<00:00, 28.95it/s]
